In [68]:
import numpy as np
import matplotlib.pyplot as plt

In [69]:
%matplotlib qt

In [108]:
# Check if HP parameters will make HP explode
alpha = 2
beta = 2
if alpha >= beta**2:  # branching ratio = ||h||L1 = alpha/beta^2
    print("Need alpha < beta^2 for the branching ratio to be < 1")

# If not, define 2nd order B-spline kernel
def kernel(t, alpha=alpha, beta=beta):
    return alpha * t * np.exp(-beta*t)

mu = 0.3

In [110]:
# Plot kernel
timevect = np.linspace(0, 10, 100)
plt.figure()
plt.plot(timevect, kernel(timevect))
plt.grid()

In [111]:
def intensity(t, past_arrivals, mu, h):
    """
    t: time at which to evaluate lambda*(t)
    past_arrivals: array containing past arrival times
    mu: background intensity
    h: excitation function

    computes lambda*(t) = mu + sum h(t-ti) for ti<t
    """
    return mu + sum(h(t - arrival) for arrival in past_arrivals if arrival < t)

### Generate a univariate HP by thinning

In [112]:
# Thinning algorithm to generate arrivals
eps = 10e-10
P = np.array([])  # array of arrivals
T = 100  # generate process on [0, T]
t = 0

while t < T:
    M = intensity(t=t+eps, past_arrivals=P, mu=mu, h=kernel)
    E = np.random.exponential(1/M)
    t = t + E
    U = np.random.uniform(low=0, high=M)
    if t < T and U <= intensity(t=t, past_arrivals=P, mu=mu, h=kernel):
        P = np.hstack([P, t])

### Plot counting process and conditional intensity

In [116]:
# Plot counting process
def plot_counting_process(P):
    """
    P: arrival times
    """
    def counting_process(arrival_times):
        return [i + 1 for i in range(len(arrival_times))]

    plt.figure()
    plt.step(x=P, y=counting_process(P), where='post', label='N(t)')
    plt.grid()
    plt.xlabel('Time')
    plt.ylabel('Count')
    plt.scatter(P, [0] * len(P), marker='x', color='red', label='Time Values')
    plt.legend(loc='upper left')

# Plot intensity function
def plot_intensity(P, T, mu, h):
    """
    P: array of arrivals
    T: arrivals contained in the interval [0, T]
    h: excitation function
    """
    # Function to calculate the sum of f(t - ti)
    def sum_excitations(t, t_values, f):
        return sum(f(t - ti) for ti in t_values if ti < t)

    time_vector = np.linspace(0, T, 500)
    intensity = mu + np.array([sum_excitations(t, P, h) for t in time_vector])

    plt.figure()
    plt.plot(time_vector, intensity, label='$\lambda^*(t)$')
    plt.grid()
    plt.xlabel('Time')
    plt.ylabel('Intensity')
    plt.scatter(P, [0] * len(P), marker='x', color='red', label='Time Values')
    plt.legend()

In [118]:
# Produce plots of counting process and conditional intensity function
plot_counting_process(P)
plot_intensity(P, T, mu=mu, h=kernel)

In [99]:
# Plot histogram of arrivals